# Classification Seeds Data Set

Na avaliação será levado em conta:
1. Corretude das tarefas solicitadas
2. Qualidade e boas práticas de codificação
3. Eficiência na manipulação dos dados

O objetivo deste quarto projeto prático da disciplina Redes Neurais Artificias é conduzir um processo de Aprendizado de Máquina com tarefa de classificação multiclasse que utilize redes neurais artificiais do tipo multilayer perceptron para endereçar esta tarefa.

Cada equipe deve produzir um Jupyter Notebook contendo boas práticas de programação que ilustrem os passos detalhados desta tarefa, com justificativas e comentários sempre que necessários.

- Kid Mendes de Oliveira Neto - 1615310011
- Wilson Calixto Cordovil Junior - 1515310026

#### Importação das Bibliotecas

In [131]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from math import sqrt
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
%matplotlib inline

#### Obtenha o seeds dataset do repositório da Universidade de California em Irvine

In [132]:
seeds_dataset = pd.read_csv('train.csv', sep=',')

#### Principais  atributo por RFE sem a limpeza dos dados

In [133]:
X_t = seeds_dataset.drop(['id','Classification'], axis=1)
Y_t = seeds_dataset['Classification']

In [134]:
modelo = ExtraTreesClassifier()#MLPClassifier
era = RFE(modelo, 6)  # número de atributos a seleccionar
era = era.fit(X_t, Y_t)

columnas = list(X_t.columns.values)
atrib = era.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
atributos

['Age', 'Glucose', 'Insulin', 'HOMA', 'Leptin', 'Resistin']

#### Removendo os outliers

In [136]:
seeds_dataset=seeds_dataset[~((seeds_dataset-seeds_dataset.mean()).abs() > 3*seeds_dataset.std())]
seeds_dataset=seeds_dataset.dropna() 

#### Avaliando os atributos mais correlacionados com o atributo Classification

In [137]:
seeds_dataset.corr(method='pearson')

,id,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
id,1.000000,0.004473,0.066726,-0.029782,0.128761,0.093959,0.009559,-0.089508,0.152130,0.343818,0.034174
Age,0.004473,1.000000,-0.044149,0.155903,-0.042690,-0.007184,-0.113997,-0.138539,-0.122520,-0.060540,-0.080083
BMI,0.066726,-0.044149,1.000000,-0.030198,0.261814,0.229877,0.652929,-0.226431,0.266711,0.309810,-0.215404
Glucose,-0.029782,0.155903,-0.030198,1.000000,0.260198,0.381024,0.036581,0.046483,-0.075517,-0.142863,0.348772
Insulin,0.128761,-0.042690,0.261814,0.260198,1.000000,0.987672,0.329513,0.147855,0.150615,0.168057,0.180136
HOMA,0.093959,-0.007184,0.229877,0.381024,0.987672,1.000000,0.304393,0.158995,0.122766,0.129263,0.207737
Leptin,0.009559,-0.113997,0.652929,0.036581,0.329513,0.304393,1.000000,-0.067030,0.315475,0.220680,-0.071767
Adiponectin,-0.089508,-0.138539,-0.226431,0.046483,0.147855,0.158995,-0.067030,1.000000,-0.148103,-0.146915,0.118205
Resistin,0.152130,-0.122520,0.266711,-0.075517,0.150615,0.122766,0.315475,-0.148103,1.000000,0.425551,0.275497
MCP.1,0.343818,-0.060540,0.309810,-0.142863,0.168057,0.129263,0.220680,-0.146915,0.425551,1.000000,-0.092911


#### Principais  atributo por pearson

Atributos que serao eliminados: MCP.1	  Adiponectin	 Leptin Age  id Classification

atributos que serao usados: BMI	Glucose	Insulin	HOMA	Resistin

#### Efetue a partição do dataset 

In [180]:
X_train = seeds_dataset.drop(['MCP.1','Adiponectin','Leptin','Age','Classification'], axis=1)

In [181]:
Y_train = seeds_dataset['Classification']

#### Principais atributo por RFE COM  a limpeza dos dados

In [182]:
modelo = ExtraTreesClassifier()#MLPClassifier
era = RFE(modelo, 6)  # número de atributos a seleccionar
era = era.fit(X_train, Y_train)

columnas = list(X_train.columns.values)
atrib = era.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
atributos

['id', 'BMI', 'Glucose', 'Insulin', 'HOMA', 'Resistin']

In [183]:
X_train.head()

,id,BMI,Glucose,Insulin,HOMA,Resistin
2,90,22.8600,82.0,4.090,0.8273,5.1400
3,8,24.2188,86.0,3.730,0.7913,10.3446
5,20,20.7600,86.0,7.553,1.6000,7.6400
6,47,26.6728,97.0,22.033,5.2718,27.8325
7,2,29.3848,90.0,4.713,1.0463,15.5562


#### Utilizar a regra da pirâmide geométrica para identificar o número de neurônios nas camadas ocultas

In [184]:
def geometric_pyramid(alpha, ni, no):
    return int(alpha * sqrt(ni*no))

#### Preparando os parâmetros o qual deve variar a função de ativação e o número de neurônios nas camadas ocultas.

In [186]:
#Quantidade de neurônios em cada camada oculta. Ex: (13,), (2,), (6,7), (9,)
hidden_layer_sizes = [

    (12, 7),
    (3, 7)
    
]

#Dicionário que servirá para a variação dos parâmetros na Rede Neural
param_grid = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'hidden_layer_sizes': hidden_layer_sizes,
    'learning_rate_init': [0.01, 0.001, 0.0005, 0.0001],
}

# MAIN

In [190]:
new_data_test = pd.read_csv('test.csv', sep=',')

mlp = MLPClassifier(activation='identity',solver='lbfgs', alpha=0.0005,hidden_layer_sizes=(3, 7), random_state=1)
mlp.fit(X_train, Y_train)
new_data_test=new_data_test.drop(['MCP.1','Adiponectin','Leptin','Age'], axis=1)


ids=new_data_test['id']
serie=mlp.predict(new_data_test)

d = { 'id': ids, 'Classification': serie}
sub = pd.DataFrame(data=d)
sub.to_csv('sub.csv', index=False)



#### Busca em grade utilizando o solver lbfgs

In [189]:
mlp = MLPClassifier(solver='lbfgs')
#grid_lbfgs = GridSearchCV(mlp, param_grid, cv=3, scoring='accuracy')
grid_lbfgs = GridSearchCV(mlp, param_grid, cv=3, scoring='f1')
grid_lbfgs.fit(X_train, Y_train)

print("Função de ativação: {}\nQuantidade de neurônios nas camadas ocultas: {}\nTaxa de Aprendizagem: {}\nMelhor acurácia: {}"
      .format(grid_lbfgs.best_params_['activation'], grid_lbfgs.best_params_['hidden_layer_sizes'],
              grid_lbfgs.best_params_['learning_rate_init'], grid_lbfgs.best_score_))

Função de ativação: identity
Quantidade de neurônios nas camadas ocultas: (3, 7)
Taxa de Aprendizagem: 0.0005
Melhor acurácia: 0.706268418956591


#### Busca em grade utilizando o solver sgd

In [147]:
mlp = MLPClassifier(solver='sgd')
grid_sgd = GridSearchCV(mlp, param_grid, cv=3, scoring='f1')
grid_sgd.fit(X_train, Y_train)

print("Função de ativação: {}\nQuantidade de neurônios nas camadas ocultas: {}\nTaxa de Aprendizagem: {}\nMelhor acurácia: {}"
      .format(grid_sgd.best_params_['activation'], grid_sgd.best_params_['hidden_layer_sizes'],
              grid_sgd.best_params_['learning_rate_init'], grid_sgd.best_score_))

Função de ativação: identity
Quantidade de neurônios nas camadas ocultas: (3, 7)
Taxa de Aprendizagem: 0.01
Melhor acurácia: 0.6956017069701281


#### Busca em grade utilizando o solver adam

In [148]:
mlp = MLPClassifier(solver='adam')
grid_adam = GridSearchCV(mlp, param_grid, cv=3, scoring='f1')
grid_adam.fit(X_train, Y_train)

print("Função de ativação: {}\nQuantidade de neurônios nas camadas ocultas: {}\nTaxa de Aprendizagem: {}\nMelhor acurácia: {}"
      .format(grid_adam.best_params_['activation'], grid_adam.best_params_['hidden_layer_sizes'],
              grid_adam.best_params_['learning_rate_init'], grid_adam.best_score_))

Função de ativação: tanh
Quantidade de neurônios nas camadas ocultas: (12, 7)
Taxa de Aprendizagem: 0.0005
Melhor acurácia: 0.7488416422287391


# Resultados